## Non Negetive Matrix Factorization for Topic Modelling

### Why should we use tf-idf vectors for NMF topic model

Using **TF-IDF** `(Term Frequency-Inverse Document Frequency)` Vectorizer with **Non-negative Matrix Factorization (NMF)** is generally recommended for a few key reasons related to how **NMF** works and what **TF-IDF** _emphasizes_. Here's why TF-IDF is typically a better choice than raw term counts when applying NMF for topic modeling:

#### 1. Emphasis on Unique and Relevant Terms
> TF-IDF assigns higher weights to terms that are more unique to each document within the entire corpus, rather than terms that are frequent across many documents.
Since NMF tries to capture distinctive, high-level features (topics) by decomposing the document-term matrix, using TF-IDF helps it focus on words that are more informative or relevant for differentiating topics. This way, common words that appear across multiple topics don't dominate the topics.
#### 2. Alignment with NMF's Matrix Decomposition
> NMF decomposes a document-term matrix into two smaller matrices (document-topic and topic-term matrices), aiming to represent each document as a combination of several key topics.
The TF-IDF weighting scheme helps balance the distribution of word frequencies so that rare but important words are not overlooked. This is particularly useful for NMF, as it performs well with the added emphasis on distinctive terms that TF-IDF provides.
#### 3. Improved Interpretability of Topics
> Using TF-IDF tends to lead to more interpretable topics because it reduces the impact of very common words that might not be specific to any topic. For instance, terms like "data" or "report" in a business-related corpus may be ubiquitous but are not necessarily informative about any specific topic. TF-IDF reduces their weight, making it easier for NMF to identify topics based on distinctive words.
As a result, the top terms in each topic produced by NMF with TF-IDF often represent cleaner, more specific themes.
#### 4. Performance in Extracting Topics from Sparse Matrices
> Document-term matrices are often very sparse, and TF-IDF can improve the quality of the matrix by reducing some of this sparsity in meaningful ways.
NMF tends to perform better with this pre-processing, as it provides a more balanced input for the factorization algorithm, allowing it to uncover latent topics more effectively than with raw counts.

In [18]:
%pip install -q pyLDAvis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

In [ ]:
import pyLDAvis
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()

In [2]:
# # Load dataset, download only 3 categories
# newsgroups = fetch_20newsgroups(subset='all', categories=['sci.space', 'rec.autos', 'talk.politics.mideast'])
# Load dataset, download ALL categories
newsgroups = fetch_20newsgroups(subset='all')
documents = newsgroups.data
print(f"{len(documents)=}")

len(documents)=18846


In [3]:
categories = newsgroups.target_names
print(f"{len(categories)=}")
print(categories)

len(categories)=20
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [12]:
# Text preprocessing with TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_features=4000, ngram_range=(1,2), min_df=10, max_df=0.3)
doc_vectors = vectorizer.fit_transform(documents)

# Apply NMF for topic modeling
n_topics = len(categories)
model = NMF(n_components=n_topics, random_state=42)
W = model.fit_transform(doc_vectors)
H = model.components_

print(f"{W.shape=}")
print(f"{H.shape=}")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


W.shape=(18846, 20)
H.shape=(20, 4000)


In [13]:
# Display key terms for each topics
key_term_map = []
num_top_words = 20
feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(H):
    key_terms = ",".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]])
    key_term_map.append(key_terms)
    print(f"Topic {topic_idx}: {key_terms}")

Topic 0: people,don,think,right,government,gun,time,israel,know,said,did,car,way,good,say,make,want,ve,years,going
Topic 1: windows,file,dos,program,window,files,thanks,help,use,graphics,using,mit,ms,problem,software,au,mail,ftp,version,pc
Topic 2: ca,game,team,games,hockey,year,players,baseball,play,season,canada,toronto,player,win,nhl,espn,league,cs,fans,roger
Topic 3: cleveland,cwru,cwru edu,freenet,cleveland freenet,freenet edu,case western,western reserve,reserve university,organization case,ins cwru,reserve,ins,western,university cleveland,case,usa lines,po cwru,po,usa
Topic 4: key,clipper,chip,encryption,keys,escrow,clipper chip,government,algorithm,security,nsa,crypto,phone,des,public,secure,use,secret,law,wiretap
Topic 5: ohio state,ohio,state,state edu,magnus,magnus acs,acs ohio,acs,state university,edu organization,university lines,ryan,cis,oil,sale,distribution,drugs,john,distribution usa,flat
Topic 6: uiuc,uiuc edu,cso,cso uiuc,illinois,university illinois,urbana,uxa,uxa c

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
# Print some documents and their topics
for i in range(5):
    topic_distribution = W[i]
    probable_topic = topic_distribution.argmax()
    print(f"Document {i+1}: {probable_topic=}  Topic Terms: {key_term_map[probable_topic]}")
    print(documents[i][:1000] + "...")  # Print a snippet of the document

    print("=" * 60)

Document 1: probable_topic=9  Topic Terms: cmu,cmu edu,andrew,andrew cmu,mellon,carnegie,carnegie mellon,pittsburgh,pittsburgh pa,pa,pa lines,mellon pittsburgh,edu subject,cs cmu,edu reply,engineering,computer,reply,pens,science
From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA
Lines: 12
NNTP-Posting-Host: po4.andrew.cmu.edu



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Visualize the Model using `pyLDAVis`

In [16]:
pyLDAvis.lda_model.prepare(model, doc_vectors, vectorizer)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.075077  0.012767       1        1  14.843594
1      0.093756  0.079244       2        1  12.670922
13     0.139890  0.009825       3        1   9.268689
16    -0.189093  0.006475       4        1   7.175119
2      0.047076 -0.029418       5        1   5.636580
4     -0.034592  0.141311       6        1   5.098814
17     0.118280 -0.083934       7        1   4.708868
8      0.065680  0.136843       8        1   4.483217
15     0.131227 -0.022213       9        1   4.251298
7      0.062903  0.048580      10        1   3.753788
11     0.034369  0.170576      11        1   3.370596
12    -0.014166  0.028922      12        1   3.312064
14    -0.248780 -0.004548      13        1   3.079409
10     0.003049  0.081356      14        1   2.855665
5      0.108694 -0.131909      15        1   2.847930
9      0.085167 -0.078551      16        1   2.811509
3     -0.016479 -0.262629      17        1   2.694477
6      0.008863 -0.037838      18        1   2.667831
18    -0.114686 -0.029676      19        1   2.311577
19    -0.206081 -0.035183      20        1   2.158053, topic_info=            Term        Freq       Total Category  logprob  loglift
3679          uk  353.000000  353.000000  Default  30.0000  30.0000
222           ac  280.000000  280.000000  Default  29.0000  29.0000
2452      netcom  246.000000  246.000000  Default  28.0000  28.0000
1776          hp  266.000000  266.000000  Default  27.0000  27.0000
224        ac uk  237.000000  237.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
775   christians   10.525182   71.675481  Topic20  -5.5120   1.9176
3526         tek    8.055303   37.940410  Topic20  -5.7795   2.2863
2981    religion   10.101310   69.050668  Topic20  -5.5531   1.9138
764     children    8.863491   57.894442  Topic20  -5.6838   1.9593
2197         mac    7.951278  112.818659  Topic20  -5.7925   1.1835

[953 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
0         1  0.026735           00
0         2  0.066837           00
0         3  0.548062           00
0         4  0.053469           00
0         5  0.106939           00
...     ...       ...          ...
3997      5  0.096625  zoo toronto
3997      8  0.386498  zoo toronto
3997     11  0.515331  zoo toronto
3998      1  0.971715         zuma
3999      1  1.016942    zuma uucp

[4304 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 14, 17, 3, 5, 18, 9, 16, 8, 12, 13, 15, 11, 6, 10, 4, 7, 19, 20])